In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/data/final_data/JEJU_MCT_KEYWORD_v2.csv")
df.head(4)

,가게명,주소,업종,이용건수구간,이용금액구간,건당평균이용금액구간,월_이용건수비중,화_이용건수비중,수_이용건수비중,목_이용건수비중,...,혼자,아이,부모님,반려동물,친척・형제,기타,주변관광지,mct_요약,keyword_요약,요약
0,비스트로낭,제주 서귀포시 안덕면 감산리 95-1번지 1층,이탈리아음식,4_50~75%,3_25~50%,2_10~25%,0.227273,0.090909,0.068182,0.181818,...,0.008772,0.166667,0.175439,0.0,0.043860,0.0,건강과성박물관,"비스트로낭의 업종은 양식, 위치는 제주 서귀포시 안덕면 감산리 95-1번지 1층 입...","비스트로낭의 가게유형은 이탈리아음식, 위치는 제주 서귀포시 안덕면 감산리 95-1번...","비스트로낭의 업종은 양식, 위치는 제주 서귀포시 안덕면 감산리 95-1번지 1층 입..."
1,예원이네,제주 서귀포시 안덕면 감산리 359-1번지 1층,"해물,생선요리",2_10~25%,1_상위 10% 이하,3_25~50%,0.197628,0.130435,0.000000,0.173913,...,0.000000,0.198473,0.221374,0.0,0.068702,0.0,건강과성박물관,"예원이네의 업종은 가정식, 위치는 제주 서귀포시 안덕면 감산리 359-1번지 1층 ...","예원이네의 가게유형은 해물,생선요리, 위치는 제주 서귀포시 안덕면 감산리 359-1...","예원이네의 업종은 가정식, 위치는 제주 서귀포시 안덕면 감산리 359-1번지 1층 ..."
2,기시어멍김밥,제주 서귀포시 강정동 232번지 1층 102호,분식,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.168950,0.000000,0.146119,0.146119,...,0.202381,0.261905,0.023810,0.0,0.047619,0.0,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ...","기시어멍김밥의 가게유형은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102...","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ..."
3,귤밭사이로수제돈가스,제주 서귀포시 강정동 587-1번지 1층,돈가스,4_50~75%,5_75~90%,5_75~90%,0.140625,0.125000,0.250000,0.156250,...,0.102564,0.179487,0.025641,0.0,0.025641,0.0,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층...","귤밭사이로수제돈가스의 가게유형은 돈가스, 위치는 제주 서귀포시 강정동 587-1번지...","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층..."


In [ ]:
df.columns

Index(['가게명', '주소', '업종', '이용건수구간', '이용금액구간', '건당평균이용금액구간', '월_이용건수비중',
       '화_이용건수비중', '수_이용건수비중', '목_이용건수비중', '금_이용건수비중', '토_이용건수비중', '일_이용건수비중',
       '05-11시_이용건수비중', '12-13시_이용건수비중', '14-17시_이용건수비중', '18-22시_이용건수비중',
       '23-04시_이용건수비중', '현지인_이용건수비중', '남성_이용건수비중', '여성_이용건수비중', '10-20대_회원비중',
       '30대_회원비중', '40대_회원비중', '50대_회원비중', '60대이상_회원비중', '상위키워드', '예약_무',
       '예약_유', '포장배달_유무', '바로 입장', '10분 이내', '30분 이내', '30분 이상', '1시간 이상',
       '2시간 이상', '데이트', '친목', '나들이', '여행', '일상', '기념일', '회식', '가족모임', '비즈니스',
       '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님', '반려동물', '친척・형제', '기타',
       '주변관광지', 'mct_요약', 'keyword_요약', '요약', '임베딩'],
      dtype='object')

In [ ]:
df['현지인_이용건수비중'].describe()

,현지인_이용건수비중
count,4191.000000
mean,0.480257
std,0.291382
min,0.000000
25%,0.189297
50%,0.524272
75%,0.757959
max,1.000000


In [ ]:
df['현지인'] = np.where(df['현지인_이용건수비중'] >= 0.48, 1, 0)
df['현지인']

,현지인
0,0
1,0
2,1
3,1
4,1
...,...
4186,0
4187,0
4188,0
4189,0


In [ ]:
df[df['현지인'] == 1]

,가게명,주소,업종,이용건수구간,이용금액구간,건당평균이용금액구간,월_이용건수비중,화_이용건수비중,수_이용건수비중,목_이용건수비중,...,아이,부모님,반려동물,친척・형제,기타,주변관광지,mct_요약,keyword_요약,요약,현지인
2,기시어멍김밥,제주 서귀포시 강정동 232번지 1층 102호,분식,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.168950,0.000000,0.146119,0.146119,...,0.261905,0.023810,0.000000,0.047619,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ...","기시어멍김밥의 가게유형은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102...","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ...",1
3,귤밭사이로수제돈가스,제주 서귀포시 강정동 587-1번지 1층,돈가스,4_50~75%,5_75~90%,5_75~90%,0.140625,0.125000,0.250000,0.156250,...,0.179487,0.025641,0.000000,0.025641,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층...","귤밭사이로수제돈가스의 가게유형은 돈가스, 위치는 제주 서귀포시 강정동 587-1번지...","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층...",1
4,광명대창집신서귀포점,제주 서귀포시 강정동 207-2번지 1층,"곱창,막창,양",6_90% 초과(하위 10% 이하),5_75~90%,2_10~25%,0.176471,0.176471,0.058824,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","광명대창집신서귀포점의 업종은 단품요리 전문, 위치는 제주 서귀포시 강정동 207-2...","광명대창집신서귀포점의 가게유형은 곱창,막창,양, 위치는 제주 서귀포시 강정동 207...","광명대창집신서귀포점의 업종은 단품요리 전문, 위치는 제주 서귀포시 강정동 207-2...",1
5,교촌치킨 서귀포혁신도시점,제주 서귀포시 강정동 166-10번지 1층,"치킨,닭강정",4_50~75%,4_50~75%,4_50~75%,0.064516,0.096774,0.112903,0.193548,...,0.368421,0.105263,0.000000,0.000000,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","교촌치킨 서귀포혁신도시점의 업종은 치킨, 위치는 제주 서귀포시 강정동 166-10번...","교촌치킨 서귀포혁신도시점의 가게유형은 치킨,닭강정, 위치는 제주 서귀포시 강정동 1...","교촌치킨 서귀포혁신도시점의 업종은 치킨, 위치는 제주 서귀포시 강정동 166-10번...",1
7,고봉민김밥인(서귀포강정점),제주 서귀포시 강정동 208-1번지 1층 102호,분식,2_10~25%,5_75~90%,6_90% 초과(하위 10% 이하),0.156250,0.000000,0.137500,0.212500,...,0.500000,0.000000,0.000000,0.000000,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","고봉민김밥인(서귀포강정점)의 업종은 분식, 위치는 제주 서귀포시 강정동 208-1번...","고봉민김밥인(서귀포강정점)의 가게유형은 분식, 위치는 제주 서귀포시 강정동 208-...","고봉민김밥인(서귀포강정점)의 업종은 분식, 위치는 제주 서귀포시 강정동 208-1번...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,효돈식(효돈식),제주 서귀포시 하효동 330-1번지 1층,한식,4_50~75%,5_75~90%,4_50~75%,0.095238,0.214286,0.119048,0.166667,...,0.146341,0.048780,0.000000,0.024390,0.024390,NaN,"효돈식(효돈식)의 업종은 가정식, 위치는 제주 서귀포시 하효동 330-1번지 1층 ...","효돈식(효돈식)의 가게유형은 한식, 위치는 제주 서귀포시 하효동 330-1번지 1층...","효돈식(효돈식)의 업종은 가정식, 위치는 제주 서귀포시 하효동 330-1번지 1층 ...",1
4181,효자동솥뚜껑 제주한림협재점,제주 제주시 한림읍 한림리 885-16번지 1층,돼지고기구이,4_50~75%,4_50~75%,4_50~75%,0.181818,0.145455,0.127273,0.163636,...,0.268908,0.092437,0.000000,0.058824,0.008403,NaN,"효자동솥뚜껑 제주한림협재점의 업종은 가정식, 위치는 제주 제주시 한림읍 한림리 88...","효자동솥뚜껑 제주한림협재점의 가게유형은 돼지고기구이, 위치는 제주 제주시 한림읍 한...","효자동솥뚜껑 제주한림협재점의 업종은 가정식, 위치는 제주 제주시 한림읍 한림리 88...",1
4182,후년하우스,제주 제주시 삼양이동 2181-19번지,샤브샤브,5_75~90%,4_50~75%,3_25~50%,0.074074,0.148148,0.148148,0.074074,...,0.181818,0.090909,0.000000,0.000000,0.000000,NaN,"후년하우스의 업종은 가정식, 위치는 제주 제주시 삼양이동 2181-19번지 입니다....","후년하우스의 가게유형은 샤브샤브, 위치는 제주 제주시 삼양이동 2181-19번지 입...","후년하우스의 업종은 가정식, 위치는 제주 제주시 삼양이동 2181-19번지 입니다....",1
4184,훔쳐온뒷고기,제주 제주시 도남동 918-9번지,돼지고기구이,6_90% 초과(하위 10% 이하),5_75~90%,2_10~25%,0.000000,0.176471,0.235294,0.176471,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,"훔쳐온뒷고기의 업종은 단품요리 전문, 위치는 제주 제주시 도남동 918-9번지 입니...","훔쳐온뒷고기의 가게유형은 돼지고기구이, 위치는 제주 제주시 도남동 918-9번지 입...","훔쳐온뒷고기의 업종은 단품요리 전문, 위치는 제주 제주시 도남동 918-9번지 입니...",1


In [ ]:
df['요약'][0]

'비스트로낭의 업종은 양식, 위치는 제주 서귀포시 안덕면 감산리 95-1번지 1층 입니다.\n주요 키워드는 파스타, 뇨끼, 분위기, 메뉴, 스테이크, 소스, 식사, 피자, 문어, 재료, 예약, 모두, 아이, 사장, 직원, 설명, 여기, 엄마, 샐러드, 양식 입니다.\n주변 관광지는 건강과성박물관 입니다.\n이용건수는 동일 업종 내 상위 50~75%이고, 현지인 이용건수 비중은 0.25로, 순위는 전체 가게 중 2588위 입니다.\n이용건수가 가장 많은 요일은 월요일(0.23), 전체 가게 중 81위 입니다. 두 번째로 많은 요일은 일요일(0.20), 전체 가게 중 204위 입니다.\n이용건수가 가장 많은 시간대는 18-22시(0.43), 전체 가게 중 779위 입니다. 두 번째로 많은 시간대는 12-13시(0.34), 전체 가게 중 612위 입니다.\n여성의 이용건수가 더 높습니다. 여성 이용건수는 전체 가맹점 중 199위(0.51)이고, 남성 이용건수는 전체 가맹점 중 325위(0.49)입니다.\n회원이 가장 많은 나이대는 30대(0.37), 전체 가게 중 93위 입니다. 두 번째로 많은 나이대는 40대(0.24), 전체 가게 중 261위 입니다.\n예약을 하고 방문한 사람들이 더 많고(0.70), 포장배달에 대해서는 매장 문의가 필요합니다.\n대부분 바로 입장(0.99), 10분 이내(0.01)의 대기시간을 가지고 입장합니다. 바로 입장의 대기시간을 가지고 방문한 손님의 비중은 전체 매장 중 3위 이고, 10분 이내의 대기시간을 가지고 방문한 손님의 비중은 전체 매장 중 774위 입니다.\n대부분 여행(0.34), 데이트(0.31) 목적으로 방문합니다. 여행 목적으로 방문한 손님의 비중은 전체 매장 중 1009위 이고, 데이트 목적으로 방문한 손님의 비중은 전체 매장 중 183위 입니다.\n연인・배우자(0.42)와 함께 가장 많이 방문하고, 부모님(0.18)와 함께 두 번째로 많이 방문합니다. 연인・배우자 동반자 유형은 전체 매장 중 393위 이고, 부모님

요약 컬럼 임베딩

In [ ]:
# 4. 임베딩 모델 로드 (Hugging Face에서 모델 가져오기)
model_name = "jhgan/ko-sroberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name, force_download=True)
model = AutoModel.from_pretrained(model_name, force_download=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [ ]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # 평균을 내어 문장 임베딩 생성
    return embeddings.squeeze().numpy()

# 요약 컬럼 임베딩
df['임베딩'] = df['요약'].apply(embed_text)


In [ ]:
df['임베딩']

,임베딩
0,"[-0.34587437, -0.03184153, -0.56805396, -0.309..."
1,"[-0.45859426, -0.03884245, -0.38955355, -0.357..."
2,"[-0.41922104, -0.03198658, -0.62383485, -0.281..."
3,"[-0.46019655, -0.085217446, -0.37959906, -0.19..."
4,"[-0.5792422, -0.058495805, -0.43737504, -0.441..."
...,...
4186,"[-0.4850063, 0.055937327, -0.47192097, -0.3517..."
4187,"[-0.42703697, 0.07083001, -0.5127611, -0.27302..."
4188,"[-0.4938826, 0.033929326, -0.41401175, -0.3246..."
4189,"[-0.4885108, -0.030345025, -0.41145855, -0.274..."


In [ ]:
df.to_csv('/content/drive/MyDrive/빅콘테스트/02.llm/예진님/data/jeju_recommend_summary.csv')

In [3]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 109.4 MB/s eta 0:00:

In [ ]:
df.columns

Index(['가게명', '주소', '업종', '이용건수구간', '이용금액구간', '건당평균이용금액구간', '월_이용건수비중',
       '화_이용건수비중', '수_이용건수비중', '목_이용건수비중', '금_이용건수비중', '토_이용건수비중', '일_이용건수비중',
       '05-11시_이용건수비중', '12-13시_이용건수비중', '14-17시_이용건수비중', '18-22시_이용건수비중',
       '23-04시_이용건수비중', '현지인_이용건수비중', '남성_이용건수비중', '여성_이용건수비중', '10-20대_회원비중',
       '30대_회원비중', '40대_회원비중', '50대_회원비중', '60대이상_회원비중', '상위키워드', '예약_무',
       '예약_유', '포장배달_유무', '바로 입장', '10분 이내', '30분 이내', '30분 이상', '1시간 이상',
       '2시간 이상', '데이트', '친목', '나들이', '여행', '일상', '기념일', '회식', '가족모임', '비즈니스',
       '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님', '반려동물', '친척・형제', '기타',
       '주변관광지', 'mct_요약', 'keyword_요약', '요약', '임베딩'],
      dtype='object')

In [ ]:
import chromadb
import os

db_folder = '/content/drive/MyDrive/빅콘테스트/03.VectorDB/mct_keyword_v3'
os.makedirs(db_folder, exist_ok=True)  # 폴더가 없을 경우 생성

# ChromaDB 클라이언트 초기화
client = chromadb.PersistentClient(path=db_folder)

# Collection 생성 (이름은 원하는 대로 설정)
collection = client.create_collection("jeju_store_mct_keyword_v4")

# 데이터 저장
for index, row in df.iterrows():
    embedding = row['임베딩'].tolist()  # 요약에 대한 임베딩 생성
    collection.add(
        ids=[f'doc_{index}'],  # 고유 ID 설정
        documents=[row['요약']],
        metadatas=[{'name': row['가게명'], 'address': row['주소'], 'industry': row['업종'], 'attraction': row['주변관광지'], 'local': row['현지인'], 'summary': row['요약']}],
        embeddings=[embedding]
    )


In [ ]:
def search_restaurants(query):
    query_embedding = embed_text(query)  # 질문 임베딩
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3  # 반환할 결과 수
    )
    return results

# 예시 질문
query = "연인과 함께 갈 만한 분위기 좋은 서귀포 맛집 추천해줘"
search_results = search_restaurants(query)

# 결과 출력
for result in search_results['metadatas'][0]:
    print(result['industry'])

국수
장어,먹장어요리
육류,고기요리


In [ ]:
df[df['가게명'] == '올레흑돼지김치말이']['요약']

,요약
1236,"올레흑돼지김치말이의 업종은 단품요리 전문, 위치는 제주 서귀포시 서귀동 276-9번..."


In [ ]:
df['요약'][1236]

'올레흑돼지김치말이의 업종은 단품요리 전문, 위치는 제주 서귀포시 서귀동 276-9번지 1층 입니다.\n주요 키워드는 김치, 돼지, 매콤, 불쇼, 올레, 소스, 고기, 시장, 서서, 양배추, 숙소, 구매, 가득, 사장, 불맛, 레시, 야시장, 식감, 포장, 저녁 입니다.\n주변 관광지는 이중섭미술관, 카사로마 호텔, 헤이, 서귀포, 골든데이지호텔서귀포오션, 서귀포 신신호텔 서귀포점, 신신호텔 천지연, 제주 엠스테이, 호텔 케니 서귀포 입니다.\n이용건수는 동일 업종 내 상위 10% 이하이고, 현지인 이용건수 비중은 0.01로, 순위는 전체 가게 중 3648위 입니다.\n이용건수가 가장 많은 요일은 토요일(0.18), 전체 가게 중 714위 입니다. 두 번째로 많은 요일은 금요일(0.17), 전체 가게 중 571위 입니다.\n이용건수가 가장 많은 시간대는 18-22시(0.58), 전체 가게 중 554위 입니다. 두 번째로 많은 시간대는 14-17시(0.34), 전체 가게 중 1138위 입니다.\n여성의 이용건수가 더 높습니다. 여성 이용건수는 전체 가맹점 중 207위(0.50)이고, 남성 이용건수는 전체 가맹점 중 317위(0.50)입니다.\n회원이 가장 많은 나이대는 30대(0.30), 전체 가게 중 161위 입니다. 두 번째로 많은 나이대는 10-20대(0.28), 전체 가게 중 215위 입니다.\n예약을 하지 않고 방문한 사람들이 더 많고(0.99), 포장배달이 가능합니다.\n대부분 바로 입장(0.44), 10분 이내(0.34)의 대기시간을 가지고 입장합니다. 바로 입장의 대기시간을 가지고 방문한 손님의 비중은 전체 매장 중 618위 이고, 10분 이내의 대기시간을 가지고 방문한 손님의 비중은 전체 매장 중 8위 입니다.\n대부분 여행(0.63), 데이트(0.18) 목적으로 방문합니다. 여행 목적으로 방문한 손님의 비중은 전체 매장 중 184위 이고, 데이트 목적으로 방문한 손님의 비중은 전체 매장 중 654위 입니다.\n연인・배우자(0.48)와 함께 가장

In [ ]:
search_results['metadatas'][0]

[{'name': '주식회사 제이콥에프앤비 무민카페제주점',
  'summary': '주식회사 제이콥에프앤비 무민카페제주점의 업종은 커피, 위치는 제주 서귀포시 안덕면 상천리 470-6번지 A동 2층 입니다.\n이용건수는 동일 업종 내 상위 10~25%이고, 현지인 이용건수 비중은 0.17입니다.\n이용건수가 가장 많은 요일은 일요일(0.23), 두 번째로 많은 요일은 토요일(0.18)입니다.\n이용건수가 가장 많은 시간대는 14-17시(0.53), 두 번째로 많은 시간대는 12-13시(0.34)입니다.\n여성의 이용건수가 더 높습니다. 남성 이용건수는 전체 가맹점 중 409위(0.45)이고, 여성 이용건수는 전체 가맹점 중 185위(0.55)입니다.\n회원이 가장 많은 나이대는 40대(0.37)이고 전체 가맹점 중 159위입니다. 회원이 두 번째로 많은 나이대는 30대(0.37)이고 전체 가맹점 중 120위입니다.'},
 {'name': '카페방주',
  'summary': '카페방주의 업종은 커피, 위치는 제주 서귀포시 안덕면 상천리 423-1번지 1층 입니다.\n이용건수는 동일 업종 내 상위 10~25%이고, 현지인 이용건수 비중은 0.07입니다.\n이용건수가 가장 많은 요일은 일요일(0.24), 두 번째로 많은 요일은 수요일(0.14)입니다.\n이용건수가 가장 많은 시간대는 12-13시(0.34), 두 번째로 많은 시간대는 14-17시(0.33)입니다.\n여성의 이용건수가 더 높습니다. 남성 이용건수는 전체 가맹점 중 423위(0.43)이고, 여성 이용건수는 전체 가맹점 중 171위(0.57)입니다.\n회원이 가장 많은 나이대는 50대(0.28)이고 전체 가맹점 중 197위입니다. 회원이 두 번째로 많은 나이대는 40대(0.26)이고 전체 가맹점 중 270위입니다.'},
 {'name': '카페루시아(본점)',
  'summary': '카페루시아(본점)의 업종은 커피, 위치는 제주 서귀포시 안덕면 창천리 840-1번지 1호 입니다.\n이용건수는 동일 업

In [ ]:
print(df[df['가맹점명'] == '카페방주']['요약'])

2266    카페방주의 업종은 커피, 위치는 제주 서귀포시 안덕면 상천리 423-1번지 1층 입...
Name: 요약, dtype: object


In [ ]:
print(df['요약'][2266])

카페방주의 업종은 커피, 위치는 제주 서귀포시 안덕면 상천리 423-1번지 1층 입니다.
이용건수는 동일 업종 내 상위 10~25%이고, 현지인 이용건수 비중은 0.07입니다.
이용건수가 가장 많은 요일은 일요일(0.24), 두 번째로 많은 요일은 수요일(0.14)입니다.
이용건수가 가장 많은 시간대는 12-13시(0.34), 두 번째로 많은 시간대는 14-17시(0.33)입니다.
여성의 이용건수가 더 높습니다. 남성 이용건수는 전체 가맹점 중 423위(0.43)이고, 여성 이용건수는 전체 가맹점 중 171위(0.57)입니다.
회원이 가장 많은 나이대는 50대(0.28)이고 전체 가맹점 중 197위입니다. 회원이 두 번째로 많은 나이대는 40대(0.26)이고 전체 가맹점 중 270위입니다.


In [4]:
import os
import chromadb
from transformers import AutoTokenizer, AutoModel
import torch

# 1. ChromaDB 데이터 저장 경로 설정 (이전에 저장했던 경로)
db_folder = '/content/drive/MyDrive/빅콘테스트/03.VectorDB/mct_keyword_v3'

# 2. PersistentClient 사용하여 기존 데이터 로드
client = chromadb.PersistentClient(path=db_folder)

# 3. 기존에 생성한 컬렉션 로드
collection = client.get_collection("jeju_store_mct_keyword_v4")

# 4. 임베딩 모델 로드 (Hugging Face에서 모델 가져오기)
model_name = "jhgan/ko-sroberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 임베딩을 위한 함수 정의
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

def search_restaurants(query):
    query_embedding = embed_text(query)  # 질문 임베딩
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5  # 반환할 결과 수
    )
    return results

# 예시 질문
query = "20대가 가장 비중이 높은 음식점은?"
search_results = search_restaurants(query)

# 결과 출력
for result in search_results['metadatas'][0]:
    print(result['name'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

20세기소년
달콤한닭강정
청년다방이도점
땡초김밥싱싱왕만두이도점
스텔라떡볶이제주아라점


In [ ]:
print(df[df['가맹점명'] == '땡초김밥싱싱왕만두이도점']['요약'])

1849    땡초김밥싱싱왕만두이도점의 업종은 분식, 위치는 제주 제주시 이도이동 1025번지 이...
Name: 요약, dtype: object


In [ ]:
df['요약'][1849]

'땡초김밥싱싱왕만두이도점의 업종은 분식, 위치는 제주 제주시 이도이동 1025번지 이튼빌딩 1층 입니다.\n이용건수는 동일 업종 내 상위 25~50%이고, 현지인 이용건수 비중은 0.61입니다.\n이용건수가 가장 많은 요일은 목요일(0.22), 두 번째로 많은 요일은 금요일(0.21)입니다.\n이용건수가 가장 많은 시간대는 14-17시(0.35), 두 번째로 많은 시간대는 18-22시(0.27)입니다.\n여성의 이용건수가 더 높습니다. 남성 이용건수는 전체 가맹점 중 400위(0.46)이고, 여성 이용건수는 전체 가맹점 중 194위(0.54)입니다.\n회원이 가장 많은 나이대는 50대(0.29)이고 전체 가맹점 중 182위입니다. 회원이 두 번째로 많은 나이대는 40대(0.29)이고 전체 가맹점 중 242위입니다.'